In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 958 kB/s 
     |████████████████████████████████| 6.4 MB 11.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import yfinance as yf
import seaborn as sns

In [4]:
"""Sobrescrevendo os métodos do pandas_datareader"""
yf.pdr_override()

In [5]:
tickers = "^BVSP ^GSPC GC=F USDBRL=X"
carteira = yf.download(tickers, interval='1mo', start="2010-01-01")["Close"]

[*********************100%***********************]  4 of 4 completed


In [6]:
carteira.columns = ["OURO", "DOLAR", "IBOV", "S&P500"]
carteira = carteira.dropna()
carteira["OURO_BRL"]=carteira["OURO"]*carteira["DOLAR"]
carteira["S&P500_BRL"]=carteira["S&P500"]*carteira["DOLAR"]

In [7]:
#retornos = carteira.pct_change().dropna()
retornos = carteira.pct_change()[1:]

In [8]:
retornos

,OURO,DOLAR,IBOV,S&P500,OURO_BRL,S&P500_BRL
Date,,,,,,
2010-02-01,0.032595,-0.049147,0.016834,0.028514,-0.018154,-0.022034
2010-03-01,-0.004471,0.018010,0.058178,0.058796,0.013459,0.077866
2010-04-01,0.060002,-0.029093,-0.040385,0.014759,0.029163,-0.014763
2010-05-01,0.027201,0.049575,-0.066385,-0.081976,0.078124,-0.036465
2010-06-01,0.027471,-0.012952,-0.033483,-0.053882,0.014163,-0.066136
...,...,...,...,...,...,...
2021-09-01,-0.031612,0.040691,-0.088850,-0.019958,0.007793,0.019921
2021-10-01,0.015781,0.040522,-0.067382,0.069144,0.056943,0.112468
2022-03-01,0.093214,-0.153363,0.159399,-0.016279,-0.074445,-0.167145


In [9]:
desvio = retornos - retornos.mean()
desvio_quadrado = desvio**2
media_desvio_quadrado = desvio_quadrado.mean()

volatilidade = np.sqrt(media_desvio_quadrado)
volatilidade

OURO          0.050459
DOLAR         0.058555
IBOV          0.072437
S&P500        0.044022
OURO_BRL      0.071403
S&P500_BRL    0.060833
dtype: float64

In [10]:
retornos.std()

OURO          0.050678
DOLAR         0.058809
IBOV          0.072751
S&P500        0.044213
OURO_BRL      0.071713
S&P500_BRL    0.061097
dtype: float64

Por que eles não combinam? Porque, por padrão, o método .std() calcula o desvio padrão da amostra, o que significa que ele usa o denominador de 𝑛−1 . Por outro lado, calculamos o desvio padrão da população, que usa um numerador de 𝑛 . Como os retornos observados são considerados amostras observadas de uma distribuição, provavelmente é mais preciso usar o denominador de 𝑛−1 , então vamos refazer nosso cálculo para ver se obtemos o mesmo número.

Para obter o número de observações, podemos usar o atributo .shape de um DataFrame que retorna uma tupla do número de linhas e colunas.

In [11]:
numero_obs = retornos.shape[0]
media_desvio_quadrado = desvio_quadrado.sum()/(numero_obs-1)
volatilidade = np.sqrt(media_desvio_quadrado)
volatilidade

OURO          0.050678
DOLAR         0.058809
IBOV          0.072751
S&P500        0.044213
OURO_BRL      0.071713
S&P500_BRL    0.061097
dtype: float64

# Volatilidade anualizada

Anualizamos a volatilidade escalando (multiplicando-a) pela raiz quadrada do número de períodos por observação

Portanto, para anualizar a volatilidade de uma série mensal, multiplicamos pela raiz quadrada de 12. Em vez de usar o `np.sqrt()`, podemos elevá-lo à potência de $0,5$

Para anualizar uma série diária, devido a variação de dias úteis por ano, podemos definir como valor padrão 240 dias (20 dias úteis mês).

In [12]:
anualizada_vol = retornos.std()*(12**0.5)
anualizada_vol

OURO          0.175553
DOLAR         0.203721
IBOV          0.252018
S&P500        0.153158
OURO_BRL      0.248420
S&P500_BRL    0.211647
dtype: float64

Agora podemos calcular os retornos mensalizados da seguinte forma:

In [13]:
#n_dias = retornos.shape[0]
#retorno_por_dia = (retornos+1).prod()**(1/n_dias) - 1
#retorno_por_dia

In [14]:
n_meses = retornos.shape[0]
retorno_mensal = (retornos+1).prod()**(1/n_meses) - 1
retorno_mensal

OURO          0.004690
DOLAR         0.008341
IBOV          0.004405
S&P500        0.011151
OURO_BRL      0.013070
S&P500_BRL    0.019585
dtype: float64

In [15]:
retorno_anual = (retorno_mensal + 1)**12-1
retorno_anual

OURO          0.057750
DOLAR         0.104814
IBOV          0.054158
S&P500        0.142337
OURO_BRL      0.168616
S&P500_BRL    0.262069
dtype: float64

In [16]:
retorno_anual/anualizada_vol

OURO          0.328958
DOLAR         0.514496
IBOV          0.214896
S&P500        0.929346
OURO_BRL      0.678755
S&P500_BRL    1.238241
dtype: float64

In [17]:
taxa_semrisco = 0.10
retorno_excesso = retorno_anual - taxa_semrisco
sharpe_ratio = retorno_excesso/anualizada_vol
sharpe_ratio

OURO         -0.240671
DOLAR         0.023629
IBOV         -0.181902
S&P500        0.276426
OURO_BRL      0.276211
S&P500_BRL    0.765755
dtype: float64